Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [122]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename).replace("\\","/"), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df = pd.DataFrame.from_records(ham)
df = pd.concat([df, pd.DataFrame.from_records(spam)])

skipped 2248.2004-09-23.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [123]:
import re

def preprocessor(e):
    e = re.sub('[^0-9a-zA-Z]+', ' ', e)
    e = e.lower()
    return e

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [124]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.
# TODO
vectorizer = CountVectorizer(preprocessor = preprocessor)

# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.
# TODO
X_train,X_test,y_train,y_test = train_test_split(df["content"],df["category"],test_size=0.25,random_state=16)


# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.
# TODO
X_train_vectorized = vectorizer.fit_transform(X_train)

# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.
# TODO
max_iter = 4000
logreg = LogisticRegression(random_state=16)

logreg.fit(X_train_vectorized, y_train)

# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.
# TODO
X_test_vectorized = vectorizer.transform(X_test)
y_pred = logreg.predict(X_test_vectorized)

# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
# TODO
print(f'Accuracy Score:\n{accuracy_score(y_test,y_pred)}\n')
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}\n')
print(f'Classification Report:\n{classification_report(y_test,y_pred)}\n')


Accuracy Score:
0.9767801857585139

Confusion Matrix:
[[902  14]
 [ 16 360]]

Classification Report:
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       916
        spam       0.96      0.96      0.96       376

    accuracy                           0.98      1292
   macro avg       0.97      0.97      0.97      1292
weighted avg       0.98      0.98      0.98      1292




Step 4.

In [126]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.
# TODO
all_features = vectorizer.get_feature_names_out()
print(all_features)

# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
# TODO
feature_importance = logreg.coef_[0]
print(feature_importance)

# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.
# TODO
feature_importance_list = list(enumerate(feature_importance))
def myFunc(e):
  return e[1]
feature_importance_list.sort(key =  myFunc, reverse = True)
for i,j in feature_importance_list[:10]:
    print(j, all_features[i])
print()
feature_importance_list.sort(key =  myFunc, reverse = False)
for i,j in feature_importance_list[:10]:
    print(j, all_features[i])



['00' '000' '0000' ... 'zzn' 'zzo' 'zzso']
[-4.68295027e-01 -3.42574213e-01  2.77229540e-03 ...  1.15385485e-06
  2.91140850e-02  6.89128426e-02]
0.8178623246216564 hello
0.809324512922452 no
0.7854958754536314 prices
0.7836569598536854 2004
0.7836361296875939 http
0.737134734759815 paliourg
0.6997586882637121 only
0.6505665211010557 remove
0.6472712835508105 removed
0.6468865944788262 more

-1.7460442492272445 attached
-1.4571913938892103 daren
-1.4452458195997324 doc
-1.3328364841001454 pictures
-1.2908924459180386 enron
-1.192037459555106 thanks
-1.084344341218914 deal
-1.08396847056904 xls
-1.0492769157911352 neon
-1.0278446100493175 meter


Submission
1. Upload the jupyter notebook to Forage.

All Done!